In [ ]:
import re
import numpy as np
from scipy.interpolate import UnivariateSpline
import pandas as pd
import tkinter as tk
from tkinter import filedialog
import matplotlib.pyplot as plt
import seaborn as sns
import kw

In [ ]:
# データの読み込み

data = kw.getdf_xlsx()['sheet1']
data.head(2)

In [ ]:
melted_df = pd.melt(data, id_vars=['TESTDATE', 'test_no'], 
                    value_vars=['capacity', 'total_head', 'current', 'output', 'shaft_power','theoretical_power', 'pump_eff'],
                    var_name='label', value_name='value')

In [ ]:
# Group by the TESTDATE, label, and test_no, and calculate the mean of the duplicate values
aggregated_df = melted_df.groupby(['TESTDATE', 'label', 'test_no'], as_index=False).mean()
aggregated_df

In [ ]:
# Pivot the aggregated DataFrame into the desired format
pivoted_df = aggregated_df.pivot(index=['TESTDATE', 'label'], columns='test_no', values='value').reset_index()

# Preview the transformed DataFrame
pivoted_df.head()

In [ ]:
# Pivot the aggregated DataFrame into the desired format
pivoted_df = aggregated_df.pivot(index=['TESTDATE', 'label'], columns='test_no', values='value').reset_index()

# Preview the transformed DataFrame
pivoted_df.head()

In [ ]:
dict = {'sheet1': pivoted_df}
kw.writedf_xlsx(dict)

In [ ]:
# Step 1: Use `melt` to convert sheet2 data to long format
melted_df2 = pivoted_df.melt(id_vars=["TESTDATE", "label"], 
                            value_vars=[1, 2, 3, 4, 5, 6], 
                            var_name="test_no", 
                            value_name="value")

In [ ]:
# Ensure the data types are consistent for further processing
melted_df2["test_no"] = melted_df2["test_no"].astype(int)

In [ ]:
# Step 2: Use `pivot` to reshape the long data into the format of sheet1
pivoted_df2 = melted_df2.pivot(index=["TESTDATE", "test_no"], 
                                     columns="label", 
                                     values="value").reset_index()
pivoted_df2.head()